## Set up

In [1]:
## Upload local script to Colab - running this creates a "choose file" button to upload local files.
from google.colab import files
uploaded = files.upload()

Saving laMap.geojson to laMap.geojson
Saving laZips.geojson to laZips.geojson
Saving starbucksInLACounty.csv to starbucksInLACounty.csv


In [2]:
## import the needed libraries
import folium
import pandas as pd
import json
from folium import plugins


## Read Data

In [3]:
## read csv
df = pd.read_csv('starbucksInLACounty.csv')

with open('laMap.geojson') as f:
    laArea = json.load(f)
# group the starbucks dataframe by zip code and count the number of stores in each zip code
numStoresSeries = df.groupby('zip').count().id

# initialize an empty dataframe to store this new data
numStoresByZip = pd.DataFrame()

# populate the new dataframe with a 'zipcode' column and a 'numStores' column
numStoresByZip['zipcode'] = [str(i) for i in numStoresSeries.index]
numStoresByZip['numStores'] = numStoresSeries.values

In [17]:
numStoresByZip

,zipcode,numStores
0,90001,2
1,90004,1
2,90007,4
3,90010,3
4,90012,6
...,...,...
236,93535,1
237,93536,4
238,93550,2
239,93551,5


## Create a map

In [24]:
# Initiatlize a new map.
laMap = folium.Map(location=[34.0522,-118.2437], tiles='cartodbpositron', zoom_start=9, attr ='<a href="https://www.openstreetmap.org/">OpenStreetMap</a> contributors, <a href="https://public.carto.com/viz/397fd294-a82b-4470-90cc-6153ebad5bf6/embed_map">Positron by Carto</a> | Data from <a href = "https://github.com/ritvikmath/StarbucksStoreScraping">Ritvik Kharkar</a>')

# ## change the legend
# bins = list(numStoresByZip["numStores"].quantile([0, 0.25, 0.5, 0.75, 1]))

# Create the choropleth map. Key components have explanatory comments.
folium.Choropleth(
    geo_data = 'laZips.geojson',         # the geojson which you want to draw on the map [in our case it is the zipcodes in LA County]
    name='choropleth',
    data= numStoresByZip,                # the pandas dataframe which contains the zipcode information and the values of the variable you want to plot on the choropleth
    columns=['zipcode', 'numStores'],    # the columns from the dataframe that you want to use
    nan_fill_color='grey',               # fill color for null values
    nan_fill_opacity=0.4,                # opacity for null values
    key_on='feature.properties.zipcode', # the common key between one of your columns and an attribute in the geojson. This is how python knows which dataframe row matches up to which zipcode in the geojson
    fill_color='YlOrRd',                 # Try some other colors: 'YlGn', 'OrRd', 'BuGn' 'BuPu', 'GnBu', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu', 'YlGn', 'YlGnBu', 'YlOrBr', 'YlOrRd'.
    fill_opacity = 0.7,                  # fill color for data values
    line_opacity = 0.2,                  # opacity for data values
    legend_name = 'Number of Starbucks',
    # bins = bins,
).add_to(laMap)

folium.LayerControl().add_to(laMap)

# Display your map
laMap

## Point Map
This map is to show the distribution of Starbucks in LA.

In [33]:
# initialize the map around LA County
laMap = folium.Map(location=[34.0522,-118.2437], tiles='Stamen Toner', zoom_start=9)

# add the shape of LA County to the map
folium.GeoJson(laArea).add_to(laMap)

# for each row in the Starbucks dataset, plot the corresponding latitude and longitude on the map
# for i,row in df.iterrows():
#     folium.CircleMarker((row.latitude,row.longitude), radius=3, weight=2, color='green', fill_color='green', fill_opacity=.5).add_to(laMap)

## use the starbuck log as the marker
for i,row in df.iterrows():
  icon = folium.features.CustomIcon("https://www.issaquahhighlands.com/wp-content/uploads/2020/04/Starbucks-250x253.png",icon_size=(14, 14))
  folium.Marker((row.latitude,row.longitude), icon=icon).add_to(laMap)
# Display your map   
laMap

## Choropleth Map
This map shows the unemployment by state using 2012 census data.

In [35]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data' #see files at https://github.com/python-visualization/folium/tree/master/examples/data
state_geo = f'{url}/us-states.json'
state_unemployment = f'{url}/US_Unemployment_Oct2012.csv'
state_data = pd.read_csv(state_unemployment)

statemap = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=state_data,
    columns=['State', 'Unemployment'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='2012 Unemployment Rate (%)'
).add_to(statemap)

folium.LayerControl().add_to(statemap)

#render your map
statemap